In [1]:
""" **Description**
    Diamondback is package which provides Digital Signal Processing ( DSP )
    solutions, organized in the form of commons, filters, interfaces, models, and
    transforms.

    Diamondback complements Artificial Intelligence ( AI ) frameworks, by
    defining components which analyze, filter, extract, model, and transform
    data into forms which are useful in applications including pattern
    recognition, feature extraction, and optimization.

    Diamondback also facilitates classical signal processing solutions
    including communications, modeling, signal identification and extraction,
    and noise cancellation.
    
    **License**
        `BSD-3C.  <https://github.com/larryturner/diamondback/blob/master/license>`_
        © 2018 - 2021 Larry Turner, Schneider Electric Industries SAS. All rights reserved.
    
    **Author**
        Larry Turner, Schneider Electric, Analytics & AI, 2018-01-23.
"""

import warnings
warnings.simplefilter( 'ignore' )
%matplotlib inline

from diamondback import Log
from diamondback import FirFilter, IirFilter, ComplexBandPassFilter, ComplexExponentialFilter
from diamondback import ComplexFrequencyFilter, DerivativeFilter, GoertzelFilter, IntegralFilter
from diamondback import PolynomialRateFilter, PolyphaseRateFilter, WindowFilter
from diamondback import DiversityModel, PrincipalComponentModel
from diamondback import ComplexTransform, FourierTransform, PowerSpectrumTransform, WaveletTransform
from PIL import Image
from typing import Any
import ipywidgets
import IPython
import math
import matplotlib    
import matplotlib.pyplot
import numpy

def label( axis : Any, title : str = '', x : str = '', y : str = '', grid : bool = True ) -> None :

    """ Label plot title and axes, autoscale, and show grid.
        
        Arguments :
            axis : Any.
            title : str.
            x : str - x-axis label.
            y : str - y-axis label.
            grid : bool.
    """

    axis.set_xlabel( x )
    axis.set_ylabel( y )
    axis.set_title( title )
    axis.grid( grid )
    axis.autoscale( enable = True, axis = 'both', tight = True )
    

In [2]:
display( IPython.display.Math( r"""\underline{Finite\;Impulse\;Response\;Filter.} \\\\
                                   \\\begin{align}
                                     & y_{n} = \sum_{i = 0}^{N} b_{i,n} x_{n-i} = \sum_{i = 1}^{N} b_{i,n} s_{i,n} + b_{0,n} x_{n} \\\\
                                     & s_{1,n+1} = x_{n}\qquad\quad s_{i,n+1} = s_{i-1,n} \\\\
                                     & H_{z,n} = \sum_{i = 0}^{N} b_{i,n} z^{-i}
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [3]:
@ipywidgets.interact( )
def interface( classification = ipywidgets.Dropdown( description = 'Classification', value = 'Hann', options = [ 'Blackman', 'Hamming', 'Hann', 'Kaiser' ], layout = { 'width' : '340px' } ),
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.1, min = 0.001, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 64, min = 1, max = 4096, step = 1, layout = { 'width' : '420px' } ),
               count = ipywidgets.IntSlider( description = 'Count', value = 1, min = 1, max = 16, step = 1, layout = { 'width' : '420px' } ),
               complement = ipywidgets.IntSlider( description = 'Complement', value = False, min = False, max = True, layout = { 'width' : '420px' } ) ) :
    
    """ Finite Impulse Response Filter.
        
        Arguments :
            classification : str - in ( 'Blackman', 'Hamming', 'Hann', 'Kaiser' ).
            frequency : float - relative to Nyquist in ( 0.0, 1.0 ).
            order : int.
            count : int.
            complement : bool.
    """

    try :
        firfilter = FirFilter.Factory.instance( FirFilter, classification, frequency, order, count, complement )
    except Exception as ex :
        Log.write( 'ERROR', ex )
        return
    widget = [ ipywidgets.Dropdown( description = r'$\vec{b_{n}}$', value = str( firfilter.b[ 0 ] ), options = [ str( x ) for x in firfilter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )[ 1 ]
    
    # Coefficients
    
    length = 1024
    n = numpy.linspace( 0.0, order, order + 1 )
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].plot( n, firfilter.b, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$', r'$\vec{b_{n}}$' )
    
    # Roots
    
    plot = [ ]
    p, z = firfilter.roots( count )
    ii = numpy.where( z.real > 10.0 )[ 0 ]
    z[ ii ] = 10.0 * numpy.sign( z[ ii ].real )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 0, 1 ].plot( p.real, p.imag, linestyle = 'None', marker = '+', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( z.real, z.imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$\vec{p_{n}}$', r'$\vec{z_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Roots$', r'$real$', r'$imag$' )
    
    # Response
    
    h, f = firfilter.response( count = count )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    ii, jj, = int( numpy.round( len( h ) // 2 * ( 1.0 - frequency ) ) ), int( numpy.round( len( h ) // 2 * ( 1.0 + frequency ) ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( -frequency, h[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 0 ].plot( frequency, h[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )
    
    # Delay
    
    d, f = firfilter.delay( count = count )
    d = numpy.round( d * 10.0 ) / 10.0  
    axis[ 1, 1 ].plot( -frequency, d[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 1 ].plot( frequency, d[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 1 ].plot( f, d, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 1, 1 ], r'$Delay$', r'$f$', r'$\tau_{f}\ n$' )
    matplotlib.pyplot.show( )
    

interactive(children=(Dropdown(description='Classification', index=2, layout=Layout(width='340px'), options=('…

In [4]:
display( IPython.display.Math( r"""\underline{Infinite\;Impulse\;Response\;Filter.} \\\\
                                   \\\begin{align}
                                     & y_{n} = \sum_{i = 1}^{N} a_{i,n} y_{n-i} + \sum_{i = 0}^{N} b_{i,n} x_{n-i} = \sum_{i = 1}^{N} (\ a_{i,n} b_{0,n} + b_{i,n}\ ) s_{i,n} + b_{0,n} x_{n}\qquad a_{0,n} = 0 \\\\
                                     & s_{1,n+1} = \sum_{i = 1}^{N} a_{i,n} s_{i,n} + x_{n}\qquad\qquad s_{i,n+1} = s_{i-1,n} \\\\
                                     & H_{z,n} = \frac{\sum_{i = 0}^{N} b_{i,n} z^{-i}}{{1 - \sum_{i = 1}^{N} a_{i,n} z^{-i}}}
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [5]:
@ipywidgets.interact( )
def interface( classification = ipywidgets.Dropdown( description = 'Classification', value = 'Butterworth', options = [ 'Bessel', 'Butterworth', 'Chebyshev' ], layout = { 'width' : '340px' } ),
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.1, min = 0.001, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 6, min = 1, max = 16, step = 1, layout = { 'width' : '420px' } ),
               count = ipywidgets.IntSlider( description = 'Count', value = 1, min = 1, max = 16, step = 1, layout = { 'width' : '420px' } ),
               complement = ipywidgets.IntSlider( description = 'Complement', value = False, min = False, max = True, layout = { 'width' : '420px' } ) ) :
    
    """ Infinite Impulse Response Filter.
        
        Arguments :
            classification : str - in ( 'Bessel', 'Butterworth', 'Chebyshev' ).
            frequency : float - relative to Nyquist in ( 0.0, 1.0 ).
            order : int.
            count : int.
            complement : bool.
    """
    
    iirfilter = IirFilter.Factory.instance( IirFilter, classification, frequency, order, count, complement )
    widget = [ ipywidgets.Dropdown( description = r'$\vec{a_{n}}$', value = str( iirfilter.a[ 0 ] ), options = [ str( x ) for x in iirfilter.a ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = r'$\vec{b_{n}}$', value = str( iirfilter.b[ 0 ] ), options = [ str( x ) for x in iirfilter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )[ 1 ]
    
    # Coefficients

    plot = [ ]
    length = 1024
    n = numpy.linspace( 0.0, order, order + 1 )    
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].bar( n, iirfilter.a, color = [ 0.8, 0.2, 0.2 ], width = 0.006 * ( order + 1 ) )
    axis[ 0, 0 ].bar( n, iirfilter.b, color = [ 0.2, 0.2, 0.8 ], width = 0.006 * ( order + 1 ) )
    plot.append( axis[ 0, 0 ].plot( n, iirfilter.a, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 0 ].plot( n, iirfilter.b, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 0 ].legend( plot, ( r'$\vec{a_{n}}$', r'$\vec{b_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$' )
    
    # Roots

    plot = [ ]
    p, z = iirfilter.roots( count )
    ii = numpy.where( z.real > 10.0 )[ 0 ]
    z[ ii ] = 10.0 * numpy.sign( z[ ii ].real )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 0, 1 ].plot( p.real, p.imag, linestyle = 'None', marker = '+', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( z.real, z.imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$\vec{p_{n}}$', r'$\vec{z_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Roots$', r'$real$', r'$imag$' )
    
    # Response
    
    h, f = iirfilter.response( count = count )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    ii, jj, = int( numpy.round( len( h ) // 2 * ( 1.0 - frequency ) ) ), int( numpy.round( len( h ) // 2 * ( 1.0 + frequency ) ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( -frequency, h[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 0 ].plot( frequency, h[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )

    # Delay
    
    d, f = iirfilter.delay( count = count )
    d = numpy.round( d * 10.0 ) / 10.0  
    axis[ 1, 1 ].plot( -frequency, d[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 1 ].plot( frequency, d[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 1 ].plot( f, d, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 1, 1 ], r'$Delay$', r'$f$', r'$\tau_{f}\ n$' )
    matplotlib.pyplot.show( )
    

interactive(children=(Dropdown(description='Classification', index=1, layout=Layout(width='340px'), options=('…

In [6]:
display( IPython.display.Math( r"""\underline{Derivative\;Filter.} \\\\
                                   \\\begin{align}
                                     & y_{n} = \sum_{i = 0}^{N} b_{i} x_{n-i} = \sum_{i = 1}^{N} b_{i} s_{i,n} + b_{0} x_{n} \\\\
                                     & s_{1,n+1} = x_{n}\qquad\quad s_{i,n+1} = s_{i-1,n} \\\\
                                     & H_{z} = \sum_{i = 0}^{N} b_{i} z^{-i} \\\\
                                     \end{align}""" ) )

display( IPython.display.Math( r"""\\\\\scriptsize{\begin{align}
                                                   & b_{1,1} = \lgroup\matrix{ 1 & -1 }\rgroup \\\\
                                                   & b_{1,2} = \lgroup\matrix{ 1 & 0 & -1 }\rgroup(\ \frac{1}{2}\ ) && b_{2,2} = \lgroup\matrix{ 1 & -2 & 1 }\rgroup \\\\
                                                   & b_{1,4} = \lgroup\matrix{ -1 & 8 & 0 & -8 & 1 }\rgroup(\ \frac{1}{12}\ ) && b_{2,4} = \lgroup\matrix{ 1 & 0 & -2 & 0 & 1 }\rgroup(\ \frac{1}{4}\ ) && b_{3,4} = \lgroup\matrix{ 1 & -2 & 0 & 2 & -1 }\rgroup \\\\
                                                   && & b_{2,6} = \lgroup\matrix{ -1 & 8 & 1 & -16 & 1 & 8 & -1 }\rgroup(\ \frac{1}{24}\ ) && b_{3,6} = \lgroup\matrix{ 1 & 0 & -3 & 0 & 3 & 0 & 1 }\rgroup(\ \frac{1}{8}\ ) \\\\
                                                   && & b_{2,8} = \lgroup\matrix{ 1 & -16 & 64 & 16 & -130 & 16 & 64 & -16 & 1 }\rgroup(\ \frac{1}{144}\ ) && b_{3,8} = \lgroup\matrix{ -1 & 8 & 2 & -24 & 0 & 24 & -2 & -8 & 1 }\rgroup(\ \frac{1}{48}\ )
                                                   \end{align}}""" ) )


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [7]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.05, 0.1 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),               
               derivative = ipywidgets.IntSlider( description = 'Derivative', value = 1, min = 1, max = 3, step = 1, layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 2, min = 0, max = 8, step = 2, layout = { 'width' : '420px' } ) ) :
    
    """ Derivative Filter.    
        
        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] relative to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            derivative : int.
            order : int.
    """
    
    order = int( max( min( order, 2.0 ** ( derivative + 1 ) ), 2.0 ** ( derivative - 1 ) ) )
    derivativefilter = DerivativeFilter.Factory.instance( DerivativeFilter, derivative, order )
    widget = [ ipywidgets.Dropdown( description = r'$\vec{b_{n}}$', value = str( derivativefilter.b[ 0 ] ), options = [ str( x ) for x in derivativefilter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )[ 1 ]

    # Coefficients
    
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    n = numpy.linspace( 0.0, order, order + 1 )
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].bar( n, derivativefilter.b, color = [ 0.2, 0.2, 0.8 ], width = 0.006 * ( order + 1 ) )
    axis[ 0, 0 ].plot( n, derivativefilter.b, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$', r'$\vec{b_{n}}$' )
    
    # Roots
    
    plot = [ ]
    p, z = derivativefilter.roots( )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 0, 1 ].plot( p.real, p.imag, linestyle = 'None', marker = '+', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( z.real, z.imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$\vec{p_{n}}$', r'$\vec{z_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Roots$', r'$real$', r'$imag$' )
    
    # Response
    
    h, f = derivativefilter.response( )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )
    
    # Filter
    
    plot = [ ]
    length = 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) ).real
    q = numpy.random.rand( len( x ) ) - 0.5
    x += q * noise / q.std( )
    y = derivativefilter.filter( x )
    n = numpy.linspace( 0, length - 1, length )
    plot.append( axis[ 1, 1 ].plot( n, x, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ 1, 1 ].plot( n, y, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    axis[ 1, 1 ].legend( plot, ( r'$x_{n}$', r'$y_{n}$' ), loc = 'upper right' )
    label( axis[ 1, 1 ], r'$Filter$', r'$n$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatRangeSlider(value=(-0.05, 0.1), description='Incident', layout=Layout(width='420px'…

In [8]:
display( IPython.display.Math( r"""\underline{Integral\;Filter.} \\\\
                                   \\\begin{align}
                                     & y_{n} = \sum_{i = 1}^{N} a_{i} y_{n-i} + \sum_{i = 0}^{N} b_{i} x_{n-i} = \sum_{i = 1}^{N} (\ a_{i} b_{0} + b_{i}\ ) s_{i,n} + b_{0} x_{n}\qquad a_{0} = 0 \\\\
                                     & s_{1,n+1} = \sum_{i = 1}^{N} a_{i} s_{i,n} + x_{n}\qquad\quad s_{i,n+1} = s_{i-1,n} \\\\
                                     & H_{z} = \frac{\sum_{i = 0}^{N} b_{i} z^{-i}}{{1 - \sum_{i = 1}^{N} a_{i} z^{-i}}} \\\\
                                     \end{align}""" ) )

display( IPython.display.Math( r"""\\\\\scriptsize{\begin{align}
                                                   & a_{1,0} = \lgroup\matrix{ 0 & 1 }\rgroup && b_{1,0} = \lgroup\matrix{ 1 }\rgroup \\\\
                                                   & a_{1,1} = \lgroup\matrix{ 0 & 1 }\rgroup && b_{1,1} = \lgroup\matrix{ 1 & 1 }\rgroup(\ \frac{1}{2}\ ) \\\\
                                                   & a_{1,2} = \lgroup\matrix{ 0 & 1 }\rgroup && b_{1,2} = \lgroup\matrix{ 1 & 4 & 1 }\rgroup(\ \frac{1}{6}\ ) \\\\
                                                   & a_{1,3} = \lgroup\matrix{ 0 & 1 }\rgroup && b_{1,3} = \lgroup\matrix{ 1 & 3 & 3 & 1 }\rgroup(\ \frac{1}{8}\ ) \\\\
                                                   & a_{1,4} = \lgroup\matrix{ 0 & 1 }\rgroup && b_{1,4} = \lgroup\matrix{ 7 & 32 & 12 & 32 & 7 }\rgroup(\ \frac{1}{90}\ )
                                                   \end{align}}""" ) )


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.05, 0.1 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 1, min = 0, max = 4, step = 1, layout = { 'width' : '420px' } ) ) :
    
    """ Integral Filter.    
        
        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] relative to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            order : int.
    """
    
    integralfilter = IntegralFilter.Factory.instance( IntegralFilter, order )
    widget = [ ipywidgets.Dropdown( description = r'$\vec{a_{n}}$', value = str( integralfilter.a[ 0 ] ), options = [ str( x ) for x in integralfilter.a ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = r'$\vec{b_{n}}$', value = str( integralfilter.b[ 0 ] ), options = [ str( x ) for x in integralfilter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )[ 1 ]
    
    # Coefficients
    
    plot = [ ]
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    n = numpy.linspace( 0.0, max( order, 1 ), max( order, 1 ) + 1 )
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].bar( n, integralfilter.a, color = [ 0.8, 0.2, 0.2 ], width = 0.006 * ( order + 1 ) )
    axis[ 0, 0 ].bar( n, integralfilter.b, color = [ 0.2, 0.2, 0.8 ], width = 0.006 * ( order + 1 ) )
    plot.append( axis[ 0, 0 ].plot( n, integralfilter.a, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 0 ].plot( n, integralfilter.b, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 0 ].legend( plot, ( r'$\vec{a_{n}}$', r'$\vec{b_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$' )

    # Roots
    
    plot = [ ]
    p, z = integralfilter.roots( )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 0, 1 ].plot( p.real, p.imag, linestyle = 'None', marker = '+', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( z.real, z.imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$\vec{p_{n}}$', r'$\vec{z_{n}}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Roots$', r'$real$', r'$imag$' )
        
    # Response
    
    h, f = integralfilter.response( )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )
    
    # Filter

    plot = [ ]
    length = 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) ).real
    q = numpy.random.rand( len( x ) ) - 0.5
    x += q * noise / q.std( )
    y = integralfilter.filter( x )
    n = numpy.linspace( 0, length - 1, length )
    plot.append( axis[ 1, 1 ].plot( n, x, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ 1, 1 ].plot( n, y, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    axis[ 1, 1 ].legend( plot, ( r'$x_{n}$', r'$y_{n}$' ), loc = 'upper right' )
    label( axis[ 1, 1 ], r'$Filter$', r'$n$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatRangeSlider(value=(-0.05, 0.1), description='Incident', layout=Layout(width='420px'…

In [10]:
display( IPython.display.Math( r"""\underline{Polynomial\;Rate\;Filter.} \\\\
                                   \\\begin{align}
                                     \end{align}""" ) )

<IPython.core.display.Math object>

In [11]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ 0.0, 0.1 ], min = -1.0, max = 0.99, step = 0.01, readout_format = '0.2f', layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),               
               rate = ipywidgets.FloatSlider( description = 'Rate', value = 1.0, min = 0.01, max = 256.0, step = 0.01, layout = { 'width' : '420px' } ) ) :

    """ Polynomial Rate Filter.    
        
        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] relative to Nyquist in [ -1.0, 1.0 ).
            noise : float - noise deviation of incident signal.
            rate : float - ratio of effective frequency in [ 1.0, inf ).
    """
    
    axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 36, 9 ) )[ 1 ]

    # Filter
    
    plot = [ ]
    ratefilter = PolynomialRateFilter( rate )
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) ).real
    q = numpy.random.rand( len( x ) ) - 0.5
    x += q * noise / q.std( )
    y = ratefilter.filter( x )
    plot.append( axis[ 0 ].plot( numpy.linspace( 0, len( x ) - 1, len( x ) ), x, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ 0 ].plot( numpy.linspace( 0, len( y ) - 1, len( y ) ), y, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    axis[ 0 ].legend( plot, ( r'$x_{n}$', r'$y_{n}$' ), loc = 'upper right' )
    label( axis[ 0 ], r'$Filter$', r'$n$' )
    axis[ 1 ].set_visible( False )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatRangeSlider(value=(0.0, 0.1), description='Incident', layout=Layout(width='420px'),…

In [12]:
display( IPython.display.Math( r"""\underline{Polyphase\;Rate\;Filter.} \\\\
                                   \\\begin{align}
                                     & b_{k,i} = b^{M}[\ k (\ N\ +\ 1\ )\ +\ i\ ] \qquad\qquad\qquad\qquad\qquad\qquad k\ :\ [\ 0,\ K\sim 256\ )\qquad\ i\ :\ [\ 0,\ N\sim 15 \ ] \\\\
                                     & y_{n} = \sum_{i = 0}^{N} b_{k_{n},i}\ x_{n-i} = \sum_{i = 1}^{N} b_{k_{n},i}\ s_{i,n} + b_{k_{n},0}\ x_{n} \\\\
                                     & s_{1,n+1} = x_{n}\qquad\ s_{i,n+1} = s_{i-1,n} \\\\                                    
                                     & \phi_{n+1,Rate} = \phi_{n,Rate}\ +\ \frac{K}{\scriptsize{Rate}} \\\\
                                     & \phi_{n+1,Rate}\ \geq\ K\qquad\longrightarrow\qquad \phi_{n+1,Rate} = \phi_{n+1,Rate}\ -\ K \\\\
                                     & k_{n+1} = \mod( \left\lfloor{\ k_{n}\ +\ \phi_{n+1,Rate}}\right\rfloor,\ M\ )
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [13]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ 0.0, 0.1 ], min = -1.0, max = 0.99, step = 0.01, readout_format = '0.2f', layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),               
               rate = ipywidgets.FloatSlider( description = 'Rate', value = 1.0, min = 0.01, max = 256.0, step = 0.01, layout = { 'width' : '420px' } ) ) :

    """ Polyphase Rate Filter.    
        
        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] relative to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            rate : float - ratio of effective frequency in ( 0.0, 256.0 ].
    """
    
    ratefilter = PolyphaseRateFilter( rate )
    widget = [ ipywidgets.Dropdown( description = 'Index', value = str( ratefilter.b.shape[ 0 ] / rate ), options = [ str( ratefilter.b.shape[ 0 ] / rate ) ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = 'Order', value = str( ratefilter.b.shape[ 0 ] ), options = [ str( ratefilter.b.shape[ 0 ] ), str( ratefilter.b.shape[ 1 ] ) ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )[ 1 ]

    # Coefficients
    
    frequency, phase, length = 0.85, numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    n = numpy.linspace( 0.0, ratefilter.b.shape[ 1 ] - 1, ratefilter.b.shape[ 1 ] )
    axis[ 0, 0 ].plot( n, numpy.zeros( ratefilter.b.shape[ 1 ] ), color = [ 0.0, 0.0, 0.0 ], linewidth = 0.5 )
    for ii in range( 0, ratefilter.b.shape[ 0 ], 4 ) :
        axis[ 0, 0 ].plot( n, ratefilter.b[ ii, : ], color = [ 0.2, 0.2, 0.8 ], linewidth = 0.25 )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$', r'$\vec{b_{n}}$' )
    
    # Filter
    
    plot = [ ]
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) ).real
    q = numpy.random.rand( len( x ) ) - 0.5
    x += q * noise / q.std( )
    y = ratefilter.filter( x )
    plot.append( axis[ 0, 1 ].plot( numpy.linspace( 0, len( x ) - 1, len( x ) ), x, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( numpy.linspace( 0, len( y ) - 1, len( y ) ), y, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$x_{n}$', r'$y_{n}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Filter$', r'$n$' )
    for ii in range( 0, ratefilter.b.shape[ 0 ], 2 ) :
        firfilter = FirFilter( ratefilter.b[ ii, : ] );
        h, f = firfilter.response( )
        h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
        ii, jj, = int( numpy.round( len( h ) // 2 * ( 1.0 - frequency ) ) ), int( numpy.round( len( h ) // 2 * ( 1.0 + frequency ) ) )
        axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 0.25 )
        d, f = firfilter.delay( )
        d = numpy.maximum( numpy.minimum( d, ratefilter.b.shape[ 1 ] // 2 + 0.6 ), ratefilter.b.shape[ 1 ] // 2 - 0.6 )
        axis[ 1, 1 ].plot( f, d, 'b', linewidth = 0.25 )        
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1, 0 ].plot( -frequency, h[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 0 ].plot( frequency, h[ jj ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    axis[ 1, 0 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )
    label( axis[ 1, 1 ], r'$Delay$', r'$f$', r'$\tau_{f}\ n$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatRangeSlider(value=(0.0, 0.1), description='Incident', layout=Layout(width='420px'),…

In [14]:
display( IPython.display.Math( r"""\underline{Complex\;Transform.} \\\\
                                   \\\begin{align}
                                     & \gamma = \left\lgroup\matrix{ 1 & \frac{-1}{2} & \frac{-1}{2}\cr 0 & \frac{3^{0.5}}{2} & \frac{-3^{0.5}}{2}\cr 1 & 1 & 1 }\right\rgroup && \alpha = \lgroup\matrix{\frac{1}{3}}\rgroup^{0.5}\ e^{\ -j\ \frac{\pi}{6}\ } \\\\
                                     & \scriptsize{Three-Phase\longrightarrow\ Complex} \\\\
                                     & y_{n} = \lgroup\matrix{ x^{T}_{n}\ \gamma\ }\rgroup \lgroup\matrix{ 1 & j & 0 }\rgroup^{T} \\\\
                                     & \overline{\scriptsize{Neutral}}\ \qquad\longrightarrow\qquad y_{n} = y_{n}\ \alpha \\\\
                                     & \scriptsize{Complex\longrightarrow\ Three-Phase} \\\\                                     
                                     & \overline{\scriptsize{Neutral}}\ \qquad\longrightarrow\qquad x_{n} = \frac{x_{n}}{\alpha} \\\\
                                     & y_{n} = \lgroup\matrix{\ \lgroup\matrix{ \gamma^{T}\gamma }\rgroup^{-1}\ \gamma^{T}\lgroup\matrix{ real(\ x_{n}\ ) & imag(\ x_{n}\ ) & 0 }\rgroup^{T}}\rgroup^{T} \\\\
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [15]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.05, 0.05 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               neutral = ipywidgets.IntSlider( description = 'Neutral', value = True, min = False, max = True, layout = { 'width' : '420px' } ),
               inverse = ipywidgets.IntSlider( description = 'Inverse', value = False, min = False, max = True, layout = { 'width' : '420px' } ) ) :

    """ Complex Transform.    
        
        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] relative to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            neutral : bool.
            inverse : bool.
    """
    
    axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )[ 1 ]

    # Incident, Transform
    
    plot = [ ]
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) )
    q = numpy.random.rand( len( x ) ) - 0.5 + 1j * ( numpy.random.rand( len( x ) ) - 0.5 )
    x += q * noise / q.std( )
    n = numpy.linspace( 0.0, length - 1, length )
    v = ComplexTransform.transform( x, neutral )
    epoch, terminus, ii, jj = r'$Incident$', r'$Transform$', 0, 1
    if ( inverse ) :
        x = ComplexTransform.transform( v, neutral )
        epoch, terminus, ii, jj = r'$Transform$', r'$Incident$', 1, 0
    plot.append( axis[ ii ].plot( n, v[ 0, : ], color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ ii ].plot( n, v[ 1, : ], color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] )
    plot.append( axis[ ii ].plot( n, v[ 2, : ], color = [ 0.8, 0.2, 0.2 ], linewidth = 1.0 )[ 0 ] )
    axis[ ii ].legend( plot, ( r'$x_{0,n}$', r'$x_{1,n}$', r'$x_{2,n}$' ), loc = 'upper right' )
    label( axis[ ii ], epoch, r'$n$' )
    plot = [ ]
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ jj ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ jj ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ jj ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ jj ].plot( x.real, x.imag, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    plot.append( axis[ jj ].plot( x[ 0 ].real, x[ 0 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ jj ].plot( x[ -1 ].real, x[ -1 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ jj ].legend( plot, ( r'$x_{0}$', r'$x_{N-1}$' ), loc = 'upper right' )
    label( axis[ jj ], terminus, r'$real$', r'$imag$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatRangeSlider(value=(-0.05, 0.05), description='Incident', layout=Layout(width='420px…

In [16]:
display( IPython.display.Math( r"""\underline{Complex\;Bandpass\;Filter.} \\\\
                                   \\\begin{align}
                                     & x_{n} = e^{\ j\ \pi\ \phi_{n}} \\\\
                                     & \phi_{n+1} = \phi_{n} + f_{n} \\\\
                                     & y_{n} = b_{n} x_{n} \\\\
                                     & e_{n} = d_{n} - y_{n} \\\\
                                     & b_{n+1} = b_{n} + \mu e_{n} x_{n}^{*}
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [17]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ 0.1, 0.1 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.1, min = -1.0, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ),
               rate = ipywidgets.FloatSlider( description = 'Rate', value = 0.01, min = 0.0001, max = 1.0, step = 0.0001, readout_format = '0.4f', layout = { 'width' : '420px' } ) ) :

    """ Complex Bandpass Filter.
        
        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] relative to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            frequency : float - relative to Nyquist in [ -1.0, 1.0 ).
            rate : float.
    """
    
    axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )[ 1 ]

    # Coefficients
    
    complexbandpassfilter = ComplexBandPassFilter( frequency, rate )
    phase, count, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 128, 1024
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) )
    q = numpy.random.rand( len( x ) ) - 0.5 + 1j * ( numpy.random.rand( len( x ) ) - 0.5 )
    x += q * noise / q.std( )
    y, e, b = complexbandpassfilter.filter( x )
    n = numpy.linspace( 0.0, length - 1, length )
    axis[ 0, 0 ].plot( n, numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].plot( n, b.real, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )
    axis[ 0, 0 ].plot( n, b.imag, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )
    axis[ 0, 0 ].plot( n, abs( b ), color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$', r'$b_{n}$' )

    # Coefficients
    
    plot = [ ]
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0, 1 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( b.real, b.imag, linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.2, 0.2, 0.8 ], markersize = 4.0 )
    plot.append( axis[ 0, 1 ].plot( b[ 0 ].real, b[ 0 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( b[ -1 ].real, b[ -1 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$b_{0}$', r'$b_{N-1}$' ), loc = 'upper right' )    
    label( axis[ 0, 1 ], r'$Coefficients$', r'$real$', r'$imag$' )
    
    # Response
    
    plot = [ ]
    h, f = numpy.zeros( count ), numpy.linspace( max( incident[ 0 ] - 0.05, -1.0 ), min( incident[ 1 ] + 0.05, 1.0 - 2.0 / count ), count )
    for ii in range( 0, len( f ) ) :
        u = ComplexExponentialFilter( phase ).filter( numpy.ones( length ) * f[ ii ] )
        complexbandpassfilter.reset( 0.0 )
        v = complexbandpassfilter.filter( u )[ 0 ]
        h[ ii ] = min( numpy.mean( abs( v[ -32 : -1 ] ) ), 1.0 )
    h, k = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) ), 20.0 * numpy.log10( numpy.maximum( abs( 1.0 - h ), numpy.finfo( float ).eps ) )
    axis[ 1, 0 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 1, 0 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1, 0 ].plot( f, k, color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )[ 0 ] )
    axis[ 1, 0 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )    
    axis[ 1, 0 ].legend( plot, ( r'$y_{f}$', r'$e_{f}$' ), loc = 'upper right' )
    label( axis[ 1, 0 ], r'$Response$', r'$f$', r'$H_{f}\ dB$' )
    
    # Filter
    
    plot = [ ]
    axis[ 1, 1 ].plot( n, x.real, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )
    axis[ 1, 1 ].plot( n, y.real, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )
    axis[ 1, 1 ].plot( n, e.real, color = [ 0.8, 0.2, 0.2 ], linewidth = 1.0 )
    plot.append( axis[ 1, 1 ].plot( n, abs( x ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1, 1 ].plot( n, abs( y ), color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1, 1 ].plot( n, abs( e ), color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )[ 0 ] )
    axis[ 1, 1 ].legend( plot, ( r'$H_{x,f}$', r'$H_{y,f}$', r'$H_{e,f}$' ), loc = 'upper right' )
    label( axis[ 1, 1 ], r'$Filter$', r'$n$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatRangeSlider(value=(0.1, 0.1), description='Incident', layout=Layout(width='420px'),…

In [18]:
display( IPython.display.Math( r"""\underline{Complex\;Frequency\;Filter.} \\\\
                                   \\\begin{align}
                                     & f_{n} = \frac{\tan^{-1}(\ b_{n}\ ) }{\pi} \\\\
                                     & x_{n} = \frac{d_{n}}{|\ d_{n}\ |} \\\\
                                     & y_{n} = b_{n} x_{n-1} \\\\
                                     & e_{n} = d_{n} - y_{n} \\\\
                                     & b_{0} = e^{\ j\ \pi\ f_{0}} \\\\
                                     & b_{n} = b_{n} + \mu e_{n} x_{n}^{*} \\\\
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [19]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.25, 0.75 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),               
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.0, min = -1.0, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ),
               rate = ipywidgets.FloatSlider( description = 'Rate', value = 0.25, min = 0.001, max = 1.0, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ) ) :

    """ Complex Frequency Filter.    
        
        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] relative to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            frequency : float - relative to Nyquist in [ -1.0, 1.0 ).
            rate : float.
    """
   
    axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )[ 1 ]

    # Coefficients
 
    plot = [ ]
    complexfrequencyfilter = ComplexFrequencyFilter( frequency, rate )
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, 1024
    f = numpy.linspace( incident[ 0 ], incident[ 1 ], length )
    x = ComplexExponentialFilter( phase ).filter( f )
    q = numpy.random.rand( len( x ) ) - 0.5 + 1j * ( numpy.random.rand( len( x ) ) - 0.5 )
    x += q * noise / q.std( )
    y, e, b = complexfrequencyfilter.filter( x )
    n = numpy.linspace( 0.0, length - 1, length )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 0.5 )
    axis[ 0 ].plot( b.real, b.imag, linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.2, 0.2, 0.8 ], markersize = 4.0 )
    plot.append( axis[ 0 ].plot( b[ 0 ].real, b[ 0 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0 ].plot( b[ -1 ].real, b[ -1 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0 ].legend( plot, ( r'$b_0$', r'$b_{N-1}$' ), loc = 'upper right' )
    label( axis[ 0 ], r'$Coefficients$', r'$real$', r'$imag$' )
    
    # Filter
    
    plot = [ ]
    axis[ 1 ].plot( n, numpy.maximum( numpy.minimum( e.real, 1.0 ), -1.0 ), color = [ 0.8, 0.2, 0.2 ], linewidth = 1.0 )
    plot.append( axis[ 1 ].plot( n, f, color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1 ].plot( n, y, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1 ].plot( n, abs( e ), color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )[ 0 ] )
    axis[ 1 ].legend( plot, ( r'$f_{x,n}$', r'$y_n$', r'$e_n$' ), loc = 'upper right' )
    label( axis[ 1 ], r'$Filter$', r'$n$', r'$f_n$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatRangeSlider(value=(-0.25, 0.75), description='Incident', layout=Layout(width='420px…

In [20]:
display( IPython.display.Math( r"""\underline{Fourier\;Transform,\;Power\;Spectrum\;Transform,\;Window\;Filter.} \\\\
                                   \\\begin{align}
                                     & y_{k} = \frac{1}{N}\ \sum_{n = 0}^{N-1} b_{n} x_{n} e^{\frac{\ -j\ \pi\ k \ n}{N}} && v_{i,k} = \frac{1}{N}\ \sum_{n = 0}^{N-1} b_{n} x_{n+i\ I} e^{ \frac{\ -j\ \pi\ k \ n}{N} } && y^{P}_{k} = \frac{1}{C}\ \sum_{i = 0}^{C-1} v_{i,k} v^{*}_{i,k} \\\\
                                     & x_{n} = \frac{N}{b_{n}}\ \sum_{k = 0}^{N-1} y_{k} e^{\frac{\ j\ \pi\ k \ n}{N}}  && n, k \in [\ 0,\ N\ )\\\\
                                     & f_{k} = -1\ + \ 2\ \frac{k}{N}\\\\
                                     & N = \frac{2}{R}
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [21]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ 0.1, 0.1 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.01, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               classification = ipywidgets.Dropdown( description = 'Classification', value = 'Hann', options = [ 'Blackman', 'Hamming', 'Hann', 'Kaiser' ], layout = { 'width' : '340px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 1023, min = 1, max = 4096, step = 1, layout = { 'width' : '420px' } ),
               normal = ipywidgets.IntSlider( description = 'Normal', value = True, min = False, max = True, layout = { 'width' : '420px' } ),              
               index = ipywidgets.IntSlider( description = 'Index', value = 64, min = 1, max = 4096, step = 1, layout = { 'width' : '420px' } ) ) :

    """ Fourier Transform, Power Spectrum Transform, Window Filter.    
        
        Arguments :
            incident : float - incident signal frequency [ epoch, terminus ] relative to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            classification : str - in ( 'Blackman', 'Hamming', 'Hann', 'Kaiser' ).
            normal : bool.
            index : int.
    """
    
    index = int( min( index, order + 1 ) )
    windowfilter = WindowFilter.Factory.instance( WindowFilter, classification, order, normal )
    indexfilter = WindowFilter.Factory.instance( WindowFilter, classification, order // 2, normal )
    widget = [ ipywidgets.Dropdown( description = r'$\vec{b_{n}}$', value = str( windowfilter.b[ 0 ] ), options = [ str( x ) for x in windowfilter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )[ 1 ]

    # Filter

    plot = [ ]
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, order + 1
    x = ComplexExponentialFilter( phase ).filter( numpy.linspace( incident[ 0 ], incident[ 1 ], length ) )
    q = numpy.random.rand( len( x ) ) - 0.5 + 1j * ( numpy.random.rand( len( x ) ) - 0.5 )
    x += q * noise / q.std( )
    n = numpy.linspace( 0.0, length - 1, length )
    h, f = FourierTransform.transform( x, windowfilter.b, False )
    k, g = PowerSpectrumTransform.transform( x, indexfilter.b, index )
    h, k = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) ), 20.0 * numpy.log10( numpy.maximum( abs( k ), numpy.finfo( float ).eps ) )
    axis[ 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    plot.append( axis[ 0 ].plot( n, windowfilter.b, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] ) 
    plot.append( axis[ 0 ].plot( n, ( windowfilter.b * x ).real, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )[ 0 ] ) 
    axis[ 0 ].plot( n, ( windowfilter.b * x ).imag, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 ) 
    axis[ 0 ].legend( plot, ( r'$\vec{b_{n}}$', r'$x_{n}$' ), loc = 'upper right' )
    label( axis[ 0 ], r'$Filter$', r'$n$' )

    # Transform
    
    plot = [ ]
    axis[ 1 ].plot( f, -3.0 * numpy.ones( len( h ) ), 'k' )
    plot.append( axis[ 1 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 1 ].plot( g, k, color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )[ 0 ] )
    axis[ 1 ].plot( numpy.linspace( incident[ 0 ], incident[ 1 ], length ), numpy.zeros( length ), color = [ 0.0, 0.0, 0.0 ], linewidth = 8.0 )    
    axis[ 1 ].legend( plot, ( r'$H^{F}_{x,f}$', r'$H^{P}_{x,f}$' ), loc = 'upper right' )
    label( axis[ 1 ], r'$Transform$', r'$f$', r'$H_{f}\ dB$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatRangeSlider(value=(0.1, 0.1), description='Incident', layout=Layout(width='420px'),…

In [22]:
display( IPython.display.Math( r"""\underline{Goertzel\;Filter,\;Window\;Filter.} \\\\
                                   \\\begin{align}
                                     & y_{n} = \sum_{i = 1}^{N} a_{i} y_{n-i} + \sum_{i = 0}^{N} b_{i} x_{n-i} = \sum_{i = 1}^{N} (\ a_{i} b_{0} + b_{i}\ ) s_{i,n} + b_{0} x_{n}\qquad a_{0} = 0 \\\\
                                     & s_{1,n+1} = \sum_{i = 1}^{N} a_{i} s_{i,n} + x_{n}\qquad\quad s_{i,n+1} = s_{i-1,n} \\\\
                                     & H_{z} = \frac{\sum_{i = 0}^{N} b_{i} z^{-i}}{{1 - \sum_{i = 1}^{N} a_{i} z^{-i}}} \\\\
                                     & N = \frac{2}{R} \\\\
                                     \end{align}""" ) )

display( IPython.display.Math( r"""\\\\\scriptsize{\begin{align}
                                                   & a = \lgroup\matrix{ 0 & 2\ \cos(\ \pi\ f\ ) & -1 }\rgroup && b = \lgroup\matrix{ 1 & -e^{\ j\ \pi\ f\ } & 0 }\rgroup \\\\
                                                   \end{align}}""" ) )


<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [23]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.FloatRangeSlider( description = 'Incident', value = [ -0.1, 0.15 ], min = -1.0, max = 0.99, step = 0.01, layout = { 'width' : '420px' } ),
               noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.0, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               classification = ipywidgets.Dropdown( description = 'Classification', value = 'Hann', options = [ 'Blackman', 'Hamming', 'Hann', 'Kaiser' ], layout = { 'width' : '340px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 1023, min = 1, max = 4096, step = 1, layout = { 'width' : '420px' } ),
               normal = ipywidgets.IntSlider( description = 'Normal', value = True, min = False, max = True, layout = { 'width' : '420px' } ),
               frequency = ipywidgets.FloatSlider( description = 'Frequency', value = 0.1, min = -1.0, max = 0.999, step = 0.001, readout_format = '0.3f', layout = { 'width' : '420px' } ) ) :

    """ Goertzel Filter, Window Filter.
        
        Arguments :        
            incident : float - incident signal frequency [ epoch, terminus ] relative to Nyquist in [ -1.0, 1.0 ).
            noise : float - incident signal deviation.
            order : int.
            normal : bool.
            frequency : float - relative to Nyquist in [ -1.0, 1.0 ).
    """
    
    windowfilter = WindowFilter.Factory.instance( WindowFilter, classification, order, normal )
    goertzelfilter = GoertzelFilter( windowfilter.b, frequency )
    widget = [ ipywidgets.Dropdown( description = r'$\vec{a_{n}}$', value = str( goertzelfilter.a[ 0 ] ), options = [ str( x ) for x in goertzelfilter.a ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = r'$\vec{b_{n}}$', value = str( goertzelfilter.b[ 0 ] ), options = [ str( x ) for x in goertzelfilter.b ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )[ 1 ]

    # Incident
    
    plot = [ ]
    phase, length = numpy.random.rand( 1 )[ 0 ] - 0.5, ( order + 1 ) * 32
    f = numpy.linspace( incident[ 0 ], incident[ 1 ], length )
    x = ComplexExponentialFilter( phase ).filter( f )
    q = numpy.random.rand( len( x ) ) - 0.5 + 1j * ( numpy.random.rand( len( x ) ) - 0.5 )
    x += q * noise / q.std( )
    h = goertzelfilter.filter( x )
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    n = numpy.linspace( 0.0, len( h ) - 1, len( h ) )
    v = numpy.exp( 1j * math.pi * numpy.linspace( 0, 2.0 * math.pi, length ) )
    axis[ 0 ].plot( [ -1.0, 1.0 ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( [ 0.0, 0.0 ], [ -1.0, 1.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( v.real, v.imag, color = [ 0.0, 0.0, 0.0 ], linewidth = 1.0 )
    axis[ 0 ].plot( x.real, x.imag, color = [ 0.2, 0.2, 0.8 ], linewidth = 1.0 )
    plot.append( axis[ 0 ].plot( x[ 0 ].real, x[ 0 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    plot.append( axis[ 0 ].plot( x[ -1 ].real, x[ -1 ].imag, linestyle = 'None', marker = 'o', markeredgecolor = [ 0.0, 0.0, 0.0 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0 ].legend( plot, ( r'$x_{0}$', r'$x_{N-1}$' ), loc = 'upper right' )
    label( axis[ 0 ], r'$Incident$', r'$real$', r'$imag$' )

    # Filter
    
    axis[ 1 ].plot( n, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 1 ], r'$Filter$', r'$n$', r'$H_{f,n}\ dB$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatRangeSlider(value=(-0.1, 0.15), description='Incident', layout=Layout(width='420px'…

In [24]:
display( IPython.display.Math( r"""\underline{Diversity\;Model.} \\\\
                                   \\\begin{align}
                                     & d_{u,v} = \left\{\begin{array} & \qquad\max(\ |\ \vec{x_{u}} - \vec{x_{v}}\ |\ ) && Chebyshev\qquad\cr\cr\cr \qquad\lgroup\matrix{\sum_{i=0}^{N}(\ \vec{x_{u,i}} - \vec{x_{v,i}}\ )^2}\rgroup^{0.5} && Euclidean\qquad\cr\cr\cr\qquad\prod_{i=0}^{N}{(\ |\ \vec{x_{u,i}} - \vec{x_{v,i}}\ |\ )}^{\frac{1}{N}} && Geometric\qquad\cr\cr\cr\qquad\sum_{i=0}^{N}{\ (\ |\ \vec{x_{u}} - \vec{x_{v}}\ |\ )\ } && Manhattan\qquad \end{array}\right. \\\\
                                     & d_{k} = \min(\ d_{u,v}\ ) & u, v \in [\ 0,\ M\ ),\ u \neq v \\\\
                                     & d_{k} \geq d_{n}\qquad \longrightarrow\qquad d_{n} = d_{k}
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [25]:
@ipywidgets.interact( )
def interface( noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.25, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ),
               classification = ipywidgets.Dropdown( description = 'Classification', value = 'Euclidean', options = [ 'Chebyshev', 'Euclidean', 'Geometric', 'Manhattan' ], layout = { 'width' : '340px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 4, min = 2, max = 64, step = 1, layout = { 'width' : '420px' } ) ) :

    """ Diversity Model.    
        
        Arguments :
            noise : float - incident signal deviation.
            classification : str - in ( 'Chebyshev', 'Euclidean', 'Geometric', 'Manhattan' ).
            order : int.
    """

    axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )[ 1 ]

    # Diversity
    
    diversitymodel = DiversityModel.Factory.instance( DiversityModel, classification, order )
    length = 1024
    n = numpy.linspace( 0.0, length - 1, length )
    x = numpy.concatenate( ( [ numpy.polyval( numpy.random.rand( 2 ), n ) ], [ numpy.polyval( numpy.random.rand( 3 ), n ) + numpy.random.rand( 1 )[ 0 ] - 0.5 ] ) )
    x = ( x.T / x.std( 1 ) ).T
    q = numpy.random.rand( x.shape[ 0 ], x.shape[ 1 ] ) - 0.5
    x += ( q.T * noise / q.std( 1 ) ).T
    d = diversitymodel.model( x )
    p = numpy.polyfit( diversitymodel.s[ 0, : ], diversitymodel.s[ 1, : ], 2 )
    y = numpy.polyval( p, x[ 0, : ] )
    axis[ 0 ].plot( n, d, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )
    label( axis[ 0 ], r'$Diversity$', r'$n$', r'$d_{n}$' )
    
    # Model
    
    plot = [ ]
    ii = numpy.argsort( x[ 0, : ] )
    plot.append( axis[ 1 ].plot( x[ 0, ii ], x[ 1, ii ], linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.2, 0.2, 0.8 ], markersize = 4.0 )[ 0 ] )
    plot.append( axis[ 1 ].plot( x[ 0, ii ], y[ ii ], linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.8, 0.2, 0.2 ], markersize = 4.0 )[ 0 ] )
    plot.append( axis[ 1 ].plot( diversitymodel.s[ 0, : ], diversitymodel.s[ 1, : ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )[ 0 ] )
    axis[ 1 ].legend( plot, ( r'$x_{n}$', r'$y_{2,n}$', r'$s_{N-1}$' ), loc = 'upper right' )
    label( axis[ 1 ], r'$Model$', r'$x_{0,n}$', r'$x_{1,n}$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatSlider(value=0.25, description='Noise', layout=Layout(width='420px'), max=0.5, step…

In [26]:
display( IPython.display.Math( r"""\underline{Principal\;Component\;Model.} \\\\
                                   \\\begin{align}
                                     & \vec{\mu_{i}} = \lgroup\matrix{\ \frac{\sum_{n=0}^{N}\vec{x_{i,n}}}{N}}\rgroup \\\\
                                     & \vec{\sigma_{i}} = \lgroup\matrix{\ \frac{\sum_{n=0}^{N}(\ \vec{x_{i,n}} - \vec{\mu_{i}}\ )^{2}}{N}}\rgroup^{0.5} \\\\
                                     & \Lambda_{n} = eig\lgroup\matrix{\ cov\lgroup\matrix{\ \lgroup\matrix{\frac{\ X_{n}^{T} - \vec{\mu}\ }{\vec{\sigma}}\ }\rgroup\ }^{T}\rgroup\ }\rgroup^{T} \\\\
                                     & Y_{n} = \Lambda_{n} \ \lgroup\matrix{\frac{\ X_{n}^{T} - \vec{\mu}\ }{\vec{\sigma}}\ }\rgroup^{T} \\\\
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [27]:
@ipywidgets.interact( )
def interface( noise = ipywidgets.FloatSlider( description = 'Noise', value = 0.05, min = 0.0, max = 0.5, step = 0.01, layout = { 'width' : '420px' } ) ) :

    """ Principal Component Model.    
        
        Arguments :
            noise : float - incident signal deviation.
    """
        
    length, valid = 1024, False
    while ( not valid ) :
        principalcomponentmodel = PrincipalComponentModel( )
        x = numpy.random.rand( 3, length ) - 0.5
        for _ in range( 0, 2 ) :
            jj = [ int( numpy.random.rand( ) * length * 0.5 ) ]
            jj += [ jj[ 0 ] + int( numpy.random.rand( ) * ( length - jj[ 0 ] - 1 ) ) + 1 ]
            jj = range( jj[ 0 ], jj[ 1 ] )
            x[ 0, jj ] += ( numpy.random.rand( ) - 0.5 ) * numpy.linspace( 0, len( jj ) - 1, len( jj ) ) * 10.0
            x[ 2, jj ] += ( numpy.random.rand( ) - 0.5 ) * x[ 0, jj ] + ( numpy.random.rand( ) - 0.5 ) * 10.0
        x = ( x.T / x.std( 1 ) ).T
        q = numpy.random.rand( x.shape[ 0 ], x.shape[ 1 ] ) - 0.5
        x += ( q.T * noise / q.std( 1 ) ).T
        x = ( x.T * ( numpy.random.rand( 3 ) + 1.0 ) + numpy.random.rand( 3 ) ).T
        y = principalcomponentmodel.model( x )
        valid = ( ( numpy.argsort( principalcomponentmodel.e ) == numpy.array( [ 1, 2, 0 ] ) ) ).all( )
    widget = [ ipywidgets.Dropdown( description = r'$\vec{\mu_{n}}$', value = str( principalcomponentmodel.u[ 0 ] ), options = [ str( x ) for x in principalcomponentmodel.u ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = r'$\vec{\lambda_{n}}$', value = str( principalcomponentmodel.e[ 0 ] ), options = [ str( x ) for x in principalcomponentmodel.e ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )    
    widget = [ ipywidgets.Dropdown( description = r'$\vec{\sigma_{n}}$', value = str( principalcomponentmodel.s[ 0 ] ), options = [ str( x ) for x in principalcomponentmodel.s ], layout = { 'width' : '340px' } ), 
               ipywidgets.Dropdown( description = r'$\Lambda_{n}$', value = '  '.join( [ f'{x:1.5f}' for x in principalcomponentmodel.v[ 0, : ] ] ), options = [ '  '.join( [ f'{x:1.5f}' for x in principalcomponentmodel.v[ ii, : ] ] ) for ii in range( 0, 3 ) ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    axis = matplotlib.pyplot.subplots( 1, 2, sharex = False, sharey = False, figsize = ( 18, 9 ) )[ 1 ]
    
    # Incident
    
    axis[ 0 ].plot( [ min( x[ 0, : ] ), max( x[ 0, : ] ) ], [ principalcomponentmodel.u[ 2 ], principalcomponentmodel.u[ 2 ] ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( [ principalcomponentmodel.u[ 0 ], principalcomponentmodel.u[ 0 ] ], [ min( x[ 2, : ] ), max( x[ 2, : ] ) ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0 ].plot( x[ 0, : ], x[ 2, : ], linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.2, 0.2, 0.8 ], markersize = 4.0 )
    axis[ 0 ].plot( [ principalcomponentmodel.u[ 0 ], principalcomponentmodel.u[ 0 ] + principalcomponentmodel.v[ 0, 0 ] ], [ principalcomponentmodel.u[ 2 ], principalcomponentmodel.u[ 2 ] + principalcomponentmodel.v[ 0, 2 ] ], color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )
    axis[ 0 ].plot( principalcomponentmodel.u[ 0 ], principalcomponentmodel.u[ 2 ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )
    axis[ 0 ].plot( principalcomponentmodel.u[ 0 ] + principalcomponentmodel.v[ 0, 0 ], principalcomponentmodel.u[ 2 ] + principalcomponentmodel.v[ 0, 2 ], linestyle = 'None', marker = 's', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )
    label( axis[ 0 ], r'$Incident$', r'$x_{0,n}$', r'$x_{2,n}$' )

    # Model
    
    axis[ 1 ].plot( [ 0.0, 0.0 ], [ min( y[ 1, : ] ) * 2.0, max( y[ 1, : ] ) * 2.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1 ].plot( [ min( y[ 0, : ] ), max( y[ 0, : ] ) ], [ 0.0, 0.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1 ].plot( [ 0.0, 0.0 ], [ min( y[ 1, : ] ) * 2.0, max( y[ 1, : ] ) * 2.0 ], color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 1 ].plot( y[ 0, : ], y[ 1, : ], linestyle = 'None', marker = 'o', markeredgecolor = 'None', markeredgewidth = 2.0, markerfacecolor = [ 0.8, 0.2, 0.2 ], markersize = 4.0 )
    label( axis[ 1 ], r'$Model$', r'$y_{0,i}$', r'$y_{1,n}$' )
    matplotlib.pyplot.show( )
    

interactive(children=(FloatSlider(value=0.05, description='Noise', layout=Layout(width='420px'), max=0.5, step…

In [28]:
display( IPython.display.Math( r"""\underline{Wavelet\;Transform.} \\\\
                                   \\\begin{align}
                                     & y_{i,0:\frac{C}{2}-1} = \downarrow(\ filter_{b_{A,L}}(\ x_{i,0:C-1}\ ),\ 2\ ) & y_{i,\frac{C}{2}:C-1} = \downarrow(\ filter_{b_{A,H}}(\ x_{i,0:C-1}\ ),\ 2\ ) && i \in [\ 0,\ R\ ) \\\\
                                     & y_{0:\frac{R}{2}-1,j} = \downarrow(\ filter_{b_{A,L}}(\ x_{0:R-1,j}\ ),\ 2\ ) & y_{\frac{R}{2}:R-1,j} = \downarrow(\ filter_{b_{A,H}}(\ x_{0:R-1,j}\ ),\ 2\ ) && j \in [\ 0,\ C\ ) \\\\
                                     & x_{0:R-1,j} = filter_{b_{S,L}}(\ 2\ \uparrow(\ y_{0:\frac{R}{2}-1,j},\ 2\ )\ ) + filter_{b_{S,H}}(\ 2\ \uparrow(\ y_{\frac{R}{2}:R-1,j},\ 2\ )\ ) &&& j \in [\ 0,\ C\ ) \\\\
                                     & x_{i,0:C-1} = filter_{b_{S,L}}(\ 2\ \uparrow(\ y_{i,0:\frac{C}{2}-1},\ 2\ )\ ) + filter_{b_{S,H}}(\ 2\ \uparrow(\ y_{i,\frac{C}{2}:C-1},\ 2\ )\ ) &&& i \in [\ 0,\ R\ )
                                     \end{align}""" ) )


<IPython.core.display.Math object>

In [29]:
@ipywidgets.interact( )
def interface( incident = ipywidgets.Dropdown( description = 'Incident', value = 'Drac Isere', options = [ 'Burning Eye', 'Drac Isere', 'Kali', 'Lionfish', 'Porsche', 'Riley' ], layout = { 'width' : '340px' } ),
               classification = ipywidgets.Dropdown( description = 'Classification', value = 'Haar', options = [ 'Coiflet', 'Daubechies', 'Haar', 'Symmlet' ], layout = { 'width' : '340px' } ),
               order = ipywidgets.IntSlider( description = 'Order', value = 1, min = 1, max = 29, step = 1, layout = { 'width' : '420px' } ),
               count = ipywidgets.IntSlider( description = 'Count', value = 2, min = 1, max = 4, step = 1, layout = { 'width' : '420px' } ),
               inverse = ipywidgets.IntSlider( description = 'Inverse', value = False, min = False, max = True, layout = { 'width' : '420px' } ) ) :

    """ Wavelet Transform.
       
        Arguments :
            incident : str - incident image in ( 'Burning Eye', Drac Isere', 'Kali', 'Lionfish', Porsche', 'Riley' ).
            classification : str - in ( 'Coiflet', 'Daubechies', 'Haar', 'Symmlet' ).
            order : int.
            count : int.
            inverse : bool.
    """
    
    _order = { 'Coiflet' : numpy.array( [ 5, 11, 17, 23, 29 ] ),
               'Daubechies' : numpy.array( [ 3, 5, 7, 9, 11, 13, 15, 17, 19 ] ),
               'Haar' : numpy.array( [ 1 ] ),
               'Symmlet' : numpy.array( [ 7, 9, 11, 13, 15, 17, 19 ] ) }
    if ( order not in _order[ classification ] ) :
        Log.write( 'ERROR', f'Classification = {classification} Order = {str( _order[ classification ] ).replace( "[", "[ " ).replace( "]", " ]" )}' )
        return
    wavelettransform = WaveletTransform.Factory.instance( WaveletTransform, classification, order )
    b = wavelettransform.b[ inverse ]
    widget = [ ipywidgets.Dropdown( description = r'$\vec{b_{L,n}}$', value = str( b[ 0 ][ 0 ] ), options = [ str( x ) for x in b[ 0 ] ], layout = { 'width' : '340px' } ),
               ipywidgets.Dropdown( description = r'$\vec{b_{H,n}}$', value = str( b[ 1 ][ 0 ] ), options = [ str( x ) for x in b[ 1 ] ], layout = { 'width' : '340px' } ) ]
    display( ipywidgets.VBox( children = widget, layout = ipywidgets.Layout( display = 'flex', flex_flow = 'row' ) ) )
    axis = matplotlib.pyplot.subplots( 2, 2, sharex = False, sharey = False, figsize = ( 18, 18 ) )[ 1 ]

    # Coefficients
    
    plot = [ ]
    x = Image.open( incident + '.png' ).convert( 'F' )
    cols, rows = x.size
    if ( ( rows % ( 2 ** count ) ) or ( cols % ( 2 ** count ) ) ) :
        Log.write( 'ERROR', f'Rows = {rows} Columns = {cols} Count = {count}' )
        return
    x = list( x.getdata( ) )
    x = numpy.array( [ x[ ii : ii + cols ] for ii in range( 0, rows * cols, cols ) ], 'float' )
    y = wavelettransform.transform( x, count, False )    
    if ( inverse ) :
        x, y = y, wavelettransform.transform( y, count, True )
        u, v = y, x
    else :
        u, v = x, y
    u[ :, : ] = ( u - u.min( ) ) * 255.0 / ( u.max( ) - u.min( ) )
    cols, rows = x.shape[ 1 ] // ( 2 ** count ), x.shape[ 0 ] // ( 2 ** count )
    q, v[ :, : ] = v[ : rows, : cols ], ( v - v[ rows :, cols : ].min( ) ) * 255.0 / ( v[ rows :, cols : ].max( ) - v[ rows :, cols : ].min( ) )
    v[ : rows, : cols ] = ( q - q.min( ) ) * 255.0 / ( q.max( ) - q.min( ) )
    n = numpy.linspace( 0, order, order + 1 )
    color = matplotlib.pyplot.get_cmap( 'gray' )
    axis[ 0, 0 ].plot( n, numpy.zeros( order + 1 ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 0 ].bar( n, b[ 0 ], color = [ 0.2, 0.2, 0.8 ], width = 0.006 * ( order + 1 ) )
    plot.append( axis[ 0, 0 ].plot( n, b[ 0 ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.2, 0.2, 0.8 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 0 ].bar( n, b[ 1 ], 0.005 * ( order + 1 ), color = [ 0.8, 0.2, 0.2 ] )
    plot.append( axis[ 0, 0 ].plot( n, b[ 1 ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = [ 1.0, 1.0, 1.0 ], markersize = 8.0 )[ 0 ] )
    axis[ 0, 0 ].legend( plot, ( r'$\vec{b_{L,n}}$', r'$\vec{b_{H,n}}$' ), loc = 'upper right' )    
    label( axis[ 0, 0 ], r'$Coefficients$', r'$n$' )
    
    # Response
    
    plot = [ ]
    h, f = FirFilter( b[ 0 ] ).response( )
    g, f = FirFilter( b[ 1 ] ).response( )    
    h = 20.0 * numpy.log10( numpy.maximum( abs( h ), numpy.finfo( float ).eps ) )
    g = 20.0 * numpy.log10( numpy.maximum( abs( g ), numpy.finfo( float ).eps ) )    
    axis[ 0, 1 ].plot( f, -3.0 * numpy.ones( len( h ) ), color = [ 0.0, 0.0, 0.0 ], linewidth = 2.0 )
    axis[ 0, 1 ].plot( [ -0.5, 0.5 ], [ -3.0, -3.0 ], linestyle = 'None', marker = 'o', markeredgecolor = [ 0.8, 0.2, 0.2 ], markeredgewidth = 2.0, markerfacecolor = 'None', markersize = 8.0 )
    plot.append( axis[ 0, 1 ].plot( f, h, color = [ 0.2, 0.2, 0.8 ], linewidth = 2.0 )[ 0 ] )
    plot.append( axis[ 0, 1 ].plot( f, g, color = [ 0.8, 0.2, 0.2 ], linewidth = 2.0 )[ 0 ] )
    axis[ 0, 1 ].legend( plot, ( r'$H_{L,f}$', r'$H_{H,f}$' ), loc = 'upper right' )
    label( axis[ 0, 1 ], r'$Response$', r'$f$', r'$H_f\ dB$' )
    
    # Incident
    
    axis[ 1, 0 ].imshow( x, cmap = color )
    label( axis[ 1, 0 ], r'$Incident$', r'$col$', r'$row$', False )

    # Transform
    
    axis[ 1, 1 ].imshow( y, cmap = color )
    label( axis[ 1, 1 ], r'$Transform$', r'$col$', r'$row$', False )
    matplotlib.pyplot.show( )
    

interactive(children=(Dropdown(description='Incident', index=1, layout=Layout(width='340px'), options=('Burnin…